<h2 style="text-align: center;">Cox model and Feature Importancy</h2>

This notebook preprocesses data, including imputing and creating a cox model for the training dataset or implementing an existing model for the test dataset.

### Choose name and data type in section 'User input'

### Results are available in the section 'Results':
- **Cox model C-index score**
- **Feature importance plot**
- **Kaplan-Meier plot**
  - Here, you can choose what features should be presented on the plot.

### Used Functions and Procedures from `/models/`:
- Preprocess data steps:
  1. Read dataframe and Create directory in `data_information/` to save data information
  2. Extract features datatype and create datatype lists from `index_list_num_cat_ord_bin.csv`

  3. Data correction:
      1. `Nein` to `Nan` for ['dx_lightrat', 'l1_lightrat', 'l2_lightrat', 'dx_lightkap', 'l1_lightkap', 'l2_lightkap', 'dx_lightlam', 'l1_lightlam', 'l2_lightlam']
      2. Convert numerical features type to float (original type: object)
      3. Correction for 'kofscore':  20.21 to Nan
      4. ECOG value 9.0 to Nan, bin category 0,1 - 0; 2,3,4 - 1
      5. `dx_sex` to `dx_female_sex` 1/0,  bin category
      6. Find outliers. Change for outliers 99-type input to Nan 
      7. BMI  
          1. Find BMI and using possible BMI interval estimate the swap weight and height columns
          2. BMI for dx, l1, l2 instead of height and weight features
          3. Calculate relative BMI change: `l1_BMI_change`, `l2_BMI_change`

      8. Data type columns:
          1. Calculate duration of rtx
          2. Calculate relative age when patient was diagnosed
          3. Drop data `ddmmyy` type columns

  4. Data mapping:
      1. Replace 'Ja'/ 'Nein'/ 'Nicht getestet' to 1/ 0/ NaN (99) numerical value
      2. Mapping object type data:
          1. Sorting by number in the line:
              Example: 
              '<3,5 mg/L': 1, '3,5 - 5,5 mg/L': 2, '>5,5 mg/L': 3
          2. Sorting by Stadium I, II, III: giving indexes of mapping 1, 2, 3
          3. For Unbekant Stadium: index 0

  5. Dropping Features
        
      'training':

          1. One unique value in column - 54 features was dropped
          2. Imbalance feature drop and categories combination with small statistic (< 6%) -> Create list of combined categories per each feature
              - Save data about categories combination to `/dictionaries/replaced_values.json`
          3. Drop all repeated/ duplicate information/ high correlated values:
              - drop_list = ['dx_weight', 'l1_weight', 'l2_weight', 'dx_height', 'l1_height','l2_height', 'l1_BMI','l2_BMI', 'l1_klinstudie','l2_klinstudie','dx_mikrogl','l1_mikrogl','l2_mikrogl','dx_albugem','l1_albugem','l2_albugem','l1_protg']
              - Drop 'l1_protg' due to high correlation with 'l2_protg'
          4. Drop outliers for ['dx_protg', 'l1_protg', 'l2_protg'] 

          after all steps, remain features list is saved in `/dictionaries/features_list_to_keep.json`
        
      'test':    

          1. Open:
          `/dictionaries/replaced_values.json`
              - to combine categories inside each feature the same way as was done for training dataset 
          `/dictionaries/features_list_to_keep.json`
              - to drop all features the same way as for training dataset
              -if some column is missing, make a warning to retrain dataset

          2. Remove outliers

  6. Preprocess data
      - 'test'
        `preprocess_test_data(df_load, additional_name='')`
          - to apply the same transformation that was learned from the training data
        
      - 'training'
        
        `preprocess_data(df_load, threshold_feature_drop=27, directory_name='models', additional_name='')`
          - drop features with high missing values: threshold_feature_drop=27
          - fixed imputers for different features categories:
              cat_imputer = SimpleImputer(strategy='most_frequent')
              num_imputer = KNNImputer(n_neighbors=10)
              cat_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') 
              scaler = QuantileTransformer()
          - split the data into training and testing sets
          - return:
              X_train = preprocessor.fit_transform(X_train) # model can learn the parameters of the transformations from this data
              X_test= preprocessor.transform(X_test) #to apply the same transformation that was learned from the training data

        `preprocess_data_choose_imputer(df_load, threshold_feature_drop=25, directory_name='models', additional_name='', cat_imputer=SimpleImputer(strategy='most_frequent'), num_imputer=KNNImputer(n_neighbors=10), scaler=QuantileTransformer())`
          - function to choose imputers for each feature categories
          - choose parameters values: 
              categorical and binomial (yes/no) feature imputer:
                  cat_imputer=SimpleImputer(strategy='most_frequent'), 
              numerical feature imputer and numerical data scaler:
                  num_imputer=KNNImputer(n_neighbors=10), 
                  scaler=QuantileTransformer()

          - split the data into training (80%) and testing (20%) sets
          - return:
              X_train = preprocessor.fit_transform(X_train) # model can learn the parameters of the transformations from this data
              X_test= preprocessor.transform(X_test) #to apply the same transformation that was learned from the training data

- Cox Model, feature importance, and Kaplan-Meier plot steps:
    1. `datatype_extraction_procedure(df, path_list="dictionaries/", report = 0)`
       procedure that returns back the lists of different types of variables from the dataset

    2. `cox_model(X_train, X_test, data_type = 'training', penalizer=0.0001, l1_ratio=0.9)`
       General Cox model

    3. `cox_feature_importancy(X_train, X_test, clean_feature = True, data_type = 'training', feature_excluded_from_plot =[], size = (30,40), directory_name='', penalizer=0.0001, l1_ratio=0.9, additional_name='' )`
       Cox model with feature importance plot.
       This function allows creating the cox model with/without Lasso less significant features clean-up method and build the plot of important features 

    4. `Kaplan_Meier_plot(X_train, lim_min_percent=10, feature_groups_specific=[], plot_name='', directory_name='')`
       Kaplan Meier curves for balanced categories

    5. `color_style()`
       color and style of plots


In [ ]:
#import all modules to the project to use functions

import sys
from pathlib import Path

# This should point to the project root where 'modules' directory is located
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# importing modules
from modules import *


In [ ]:
import numpy as np
import pandas as pd
import re
import os
from datetime import datetime

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, PowerTransformer, Normalizer, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


Set_up_option

# 1.0 User input  
### Choose information about dataset

In [ ]:
#Choose 1/0 train/test data type
data_type_train = 1  #1 - train,  0 - test

In [ ]:
#Correct data_name in case of the different data type names

#it choose data type: data_frame_type = training or test  
#it choose data file name: data_name

if data_type_train ==1:
    #training data
    data_name = 'training_df'  
    data_frame_type = 'training' 

elif data_type_train ==0:
    #test data
    data_name = 'test_df'
    data_frame_type = 'test'

******************************************************** Automatical section *****************************************************************************
# 2. Data preprocessing

### 2.1 read data file:

In [ ]:

df = read_df(data_name)

### 2.2 Create directory to save information about data

In [ ]:
#def create_directory_main(data_name, parent_directory = "intermediate_report")
directory_name = create_directory_main(data_name, parent_directory = "intermediate_report")

### 2.3 Correct data
1. Convert "Nein" to `NaN` for the following columns:
   - `['dx_lightrat', 'l1_lightrat', 'l2_lightrat', 'dx_lightkap', 'l1_lightkap', 'l2_lightkap', 'dx_lightlam', 'l1_lightlam', 'l2_lightlam']`
2. Convert numerical features type to float (original type: object).
3. Correction for `'kofscore'`: replace `20.21` with `NaN`.
4. ECOG value `9.0` to `NaN`, bin category `0,1` - `0`; `2,3,4` - `1`.
5. Convert `dx_sex` to `dx_female_sex` `1/0`, bin category.
6. Find outliers. Change for outliers `99`-type input to `NaN`.
7. BMI:
    1. Find BMI and using possible BMI interval estimate the swap weight and height columns.
    2. Implement BMI for `dx`, `l1`, `l2` instead of height and weight features.
    3. Calculate relative BMI change: `l1_BMI_change`, `l2_BMI_change`.
8. Data type columns:
    1. Calculate duration of rtx.
    2. Calculate relative age when the patient was diagnosed.
    3. Drop data `ddmmyy` type columns.


In [ ]:
df = data_correction(df)

### 2.4 Save information about amount of missing and unique values for each feature

In [ ]:
df_missing_count(df,directory_name)

### 2.5 Data Mapping

1. Replace categorical values with numerical values:
   - Convert `'Ja'`/ `'Nein'`/ `'Nicht getestet'` to `1`/ `0`/ `NaN` (for `99`) respectively.
   
2. Mapping object type data:
   1. Sorting by number in the line:
      - Example: 
        - `'<3,5 mg/L'`: 1
        - `'3,5 - 5,5 mg/L'`: 2
        - `'>5,5 mg/L'`: 3
   2. Sorting by Stadium:
      - Assign indexes based on stadium number:
        - `Stadium I`: 1
        - `Stadium II`: 2
        - `Stadium III`: 3
   3. For `Unbekannt Stadium` (Unknown Stadium):
      - Assign index 0.


In [ ]:
df_02_mapping, value_mapping = data_mapping(df, directory_name, data_name)

In [ ]:
df_02_mapping.head()

output mapping for values for each features:

In [ ]:
value_mapping

### 2.6 Drop features

**'training':**

1. **One unique value in column:**
   - 54 features were dropped due to having only one unique value.
   
2. **Imbalance feature drop and categories combination with small statistics (< 6%): combine_categories=True**
   - Create a list of combined categories per each feature.
   - Save data about categories combination to `'helper/data_dictionary/replaced_values.json'`.
   
3. **Drop all repeated/duplicate information/high correlated values:**
   - `drop_list = ['dx_weight', 'l1_weight', 'l2_weight', 'dx_height', 'l1_height','l2_height', 'l1_BMI','l2_BMI', 'l1_klinstudie','l2_klinstudie','dx_mikrogl','l1_mikrogl','l2_mikrogl','dx_albugem','l1_albugem','l2_albugem','l1_protg']`
   - Drop `'l1_protg'` due to high correlation with `'l2_protg'`.
   
4. **Drop outliers for `['dx_protg', 'l1_protg', 'l2_protg']`.**

   - After all steps, the remaining features list is saved in `'helper/data_dictionary/features_list_to_keep.json'`.

**'test':**

1. **Open:**
   - `'helper/data_dictionary/replaced_values.json'` to combine categories inside each feature the same way as was done for the training dataset.
   - `'helper/data_dictionary/features_list_to_keep.json'` to drop all features the same way as for the training dataset. If some column is missing, make a warning to retrain the dataset.

2. **Remove outliers.**


In [ ]:
#def drop_features(df,min_percent = 0.06, data_type ='train', combine_categories=True, data_name='', directory_name='', drop_list = ['dx_weight', 'l1_weight', 'l2_weight', 'dx_height', 'l1_height','l2_height',	'l1_BMI'	,'l2_BMI', 'l1_klinstudie','l2_klinstudie','dx_mikrogl','l1_mikrogl','l2_mikrogl','dx_albugem','l1_albugem','l2_albugem','l1_protg']):
df_02_mapping_drop =  drop_features(df_02_mapping,min_percent = 0.06, data_type =data_frame_type, combine_categories=True, data_name=data_name, directory_name=directory_name, drop_list = ['dx_weight', 'l1_weight', 'l2_weight', 'dx_height', 'l1_height','l2_height',	'l1_BMI'	,'l2_BMI', 'l1_klinstudie','l2_klinstudie','dx_mikrogl','l1_mikrogl','l2_mikrogl','dx_albugem','l1_albugem','l2_albugem','l1_protg'])
 

In [ ]:
df_02_mapping_drop.shape

### 2.7 Check unique sets of drugs. 
Could be used to combine drugs by sets in the analysis

In [ ]:
l_list = ['l2_', 'l1_']

for l_element in l_list:
    print(f'l_element: {l_element}')

    columns_containing_drug = [column for column in df_02_mapping_drop.columns if 'drug' in column.lower()]
    df_02_mapping_drugs = df_02_mapping_drop[columns_containing_drug]
    l_columns = [col for col in df_02_mapping_drugs.columns if col.startswith(l_element)]

    # Initialize a list to hold the results for each row
    results = []

    # Iterate over each row in the DataFrame
    for index, row in df_02_mapping.iterrows():
        # Initialize a temporary list A for this row
        A = []
        
        # Check each 'l_' column in this row
        for col in l_columns:
            if row[col] == 1:
                A.append(col)
        
        # Add the list A to the results (one list per row)
        results.append(A)

    # Count the occurrences of each unique subset
    unique_subset_counts = {}
    for sublist in results:
    # Sort and convert to tuple for dictionary key
        sublist_tuple = tuple(sorted(sublist))
        if sublist_tuple in unique_subset_counts:
            unique_subset_counts[sublist_tuple] += 1
        else:
            unique_subset_counts[sublist_tuple] = 1

    # Sort the subsets based on their counts
    sorted_subsets = sorted(unique_subset_counts.items(), key=lambda x: x[1], reverse=True)

    # Print the sorted subsets with their counts
    for subset, count in sorted_subsets:
        print(f"Subset: {list(subset)}, Count: {count}")

    print(f'Number of unique subsets: {len(unique_subset_counts)}')


In [ ]:
''' #code in process to combine subsets
#test
l_element=[['l2_drug_1', 'l2_drug_3'], ['l2_drug_3', 'l2_drug_6'], ['l2_drug_3', 'l2_drug_6'],['l2_drug_1', 'l2_drug_3', 'l2_drug_3', 'l2_drug_6'], ['l2_drug_3', 'l2_drug_6']]
set_list = [set(sublist) for sublist in l_element]
combined_subsets = set()
for i, subset_i in enumerate(set_list):
    for j, subset_j in enumerate(set_list):
        if i != j and subset_i.issubset(subset_j):
            combined_subsets.add(tuple(l_element[i]))

# Convert the set of tuples back to a list of lists
combined_subsets = [list(subset) for subset in combined_subsets]

combined_subsets'''

# 3. RESULTS

### 3.1 Data Preprocess , Feature importancy plot 

In [ ]:
if data_frame_type=='training':
    X_train, X_test = preprocess_data(df_02_mapping_drop, threshold_feature_drop=27)

#def cox_feature_importancy(X_train, X_test, clean_feature = True ,data_type = 'training', feature_excluded_from_plot =[],size = (30,40), directory_name='',penalizer=0.0001, l1_ratio=0.9 ):

    top_f_df, score_X_train, score_X_test, cph = cox_feature_importancy(X_train,X_test, clean_feature =True,data_type = data_frame_type, feature_excluded_from_plot =['bin__l2_rtpnotwnierenin','bin__l1_crabkritb','bin__dx_crabkritb','cat__l2_rtptheraentsch_3.0','cat__l2_prottyp_1.0','cat__l1_prottyp_2.0'],size = (30,40), directory_name=directory_name, penalizer=0.001, l1_ratio=0.9)
    print(score_X_train, score_X_test)

elif data_frame_type=='test':
    df_test = preprocess_test_data(df_02_mapping_drop)
    top_f_df, score_X_train, score_X_test, cph = cox_feature_importancy(_,df_test, clean_feature =True,data_type = data_frame_type,feature_excluded_from_plot =['bin__l2_rtpnotwnierenin','bin__l1_crabkritb','bin__dx_crabkritb','cat__l2_rtptheraentsch_3.0','cat__l2_prottyp_1.0','cat__l1_prottyp_2.0'],size = (50,50), directory_name=directory_name, penalizer=0.001, l1_ratio=0.9)
   

### 3.2 C-index Evaluation

The Concordance index (C-index) is a measure used to evaluate the predictive accuracy of survival models. It assesses the model's ability to correctly rank pairs of subjects based on their predicted survival times. The C-index is calculated for both the training and test datasets, with specific considerations for each:

- **Training Dataset:**
  - **Train Set:** Calculate the C-index on the training portion of the dataset to evaluate the model's fit and its ability to predict outcomes on data it has already seen.
  - **Test Set:** Calculate the C-index on the test portion of the training dataset to assess the model's generalizability and its predictive accuracy on unseen data.  
  

- **Test Dataset:**
  - **Test:** For the external or separate test dataset, calculate the C-index to further validate the model's predictive performance on new, unseen data. This step is crucial for understanding how the model performs in real-world scenarios or when applied to data from different sources or time periods.

The C-index values obtained from these evaluations provide insights into the model's predictive accuracy and generalizability, guiding improvements and adjustments to enhance performance.



In [ ]:
score_X_train, score_X_test

### 3.3 Results of another cox model without Lasso less significant features clean up method 

Just to control the difference in score and effectivness of Lasso less significant features clean up method 

In [ ]:
if data_frame_type=='training':
    score_X_train2,score_X_test2 = cox_model(X_train,X_test,data_type = data_frame_type, penalizer=0.001, l1_ratio=0.7)
elif data_frame_type=='test':
    score_X_train2,score_X_test2 = cox_model(_,df_test,data_type = data_frame_type, penalizer=0.001, l1_ratio=0.7)

In [ ]:
score_X_train2, score_X_test2

### 3.4 Kaplan-Meier Plot

The Kaplan-Meier plot is a non-parametric statistic used to estimate the survival function from lifetime data. To customize the Kaplan-Meier plot based on specific features, follow the steps below:

1. **Choose Features to be Shown on Plot:**
   - Identify and select the features you wish to visualize in the Kaplan-Meier plot. Group these features into a list named `feature_groups_specific`. This allows for a focused analysis on the impact of these specific features on survival.
   
2. **Plot Name:**
   - Assign a meaningful name to your plot for easy identification. Store this name in a variable called `plot_name`. This name will be used as the title of the Kaplan-Meier plot or for reference purposes in your analysis.




In [ ]:
if data_frame_type=='test':
    X_train=df_test

In [ ]:
#Kaplan_Meier_plot(X_train,lim_min_percent=10,feature_groups_specific=[], plot_name='', directory_name=''):
Kaplan_Meier_plot(X_train,lim_min_percent=10,feature_groups_specific = ['bin__l2_drug_5','bin__l1_drug_6','bin__l2_drug_4'],plot_name='Drugs', directory_name=directory_name)

In [ ]:

Kaplan_Meier_plot(X_train,lim_min_percent=10,feature_groups_specific = ['cat__dx_einrid_1.0','cat__dx_einrid_3.0'],plot_name='Clinic', directory_name=directory_name)